# Fallback - Product Propensity Scoring
* StellarAlgo Data Science
* Ryan Kazmerik
* Oct 19, 2022

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pycaret.classification import *
from shared_utilities import helpers

from importlib import reload

from shared_utilities import helpers
reload(helpers)

<module 'shared_utilities.helpers' from '/Users/ryankazmerik/Git/data-sci/shared_utilities/shared_utilities/helpers.py'>

In [2]:
CLUSTER = "prod-app"
DATABASE = "stlrcanucks"
LKUPCLIENTID = "7"
SCORING_YEAR = 2022

In [8]:
df = helpers.get_product_propensity_dataset(
    cluster=CLUSTER,
    database=DATABASE,
    lkupclientid=LKUPCLIENTID,
    scoring_year=SCORING_YEAR,
    type_flag=0
)

df.shape

Authorized as AROASQ4JELIXYLYV6P4UV:rkazmerik@stellaralgo.com


(699170, 25)

In [9]:
df.head()

,lkupclientid,clientcode,dimcustomermasterid,seasonyear,distance,tenure,events_prior,attended_prior,events_last,attended_last,atp_last,product_last,product_current,product,spend_current,volume_current,attendance_current,events_current,sends,sends_prior,opens,opens_prior,date_last_send,date_last_touch,date_last_save
0,7,CANUCKS,5575,2018,0.39,-1,1,1,0.0,0.0,0.0,None,None,None,131.25,4,1.00000000000000000,1,NaN,NaN,NaN,NaN,None,None,2022-08-17
1,7,CANUCKS,5580,2021,29.33,-1,1,1,0.0,0.0,0.0,None,Individual,None,177.00,2,1.00000000000000000,1,NaN,NaN,NaN,NaN,None,None,2022-10-19
2,7,CANUCKS,5581,2019,9.13,-1,1,1,0.0,0.0,0.0,None,Individual,None,313.00,2,1.00000000000000000,1,NaN,NaN,NaN,NaN,None,None,2022-10-18
3,7,CANUCKS,5581,2021,9.13,-1,1,1,1.0,1.0,156.5,Individual,None,None,0.00,0,0E-17,0,NaN,NaN,NaN,NaN,None,None,2022-10-18
4,7,CANUCKS,5583,2019,3.07,-1,1,1,0.0,0.0,0.0,None,None,None,192.66,2,1.00000000000000000,1,1.0,2.0,2.0,1.0,2022-08-31,None,2022-10-18


In [10]:
# choose the features for the stellar base PP model
features = [
    "atp_last",
    "attendance_current",
    "attended_last",
    "attended_prior",
    "clientcode",
    "dimcustomermasterid",
    "date_last_save",
    "date_last_send",
    "date_last_touch",
    "events_last",
    "events_prior",
    "distance",
    "events_current",
    "lkupclientid",
    "opens",
    "opens_prior",
    "product",
    "product_current",
    "product_last",
    "seasonyear",
    "sends",
    "sends_prior",
    "spend_current",
    "tenure",
    "volume_current"
]

# copy your main dataframe
df_dataset = df

#creating the training and evaluation dataframe
df_dataset = df_dataset[features]

df_train = df_dataset.sample(frac=0.70, random_state=786)
df_eval = df_dataset.drop(df_train.index)

df_train.reset_index(drop=True, inplace=True)
df_eval.reset_index(drop=True, inplace=True)

# print out the number of records for training and eval
print('Data for Modeling: ' + str(df_train.shape))
print('Unseen Data For Predictions: ' + str(df_eval.shape), end="\n\n")

Data for Modeling: (489419, 25)
Unseen Data For Predictions: (209751, 25)



In [11]:
setup(
    data= df_train, 
    target="product", 
    train_size = 0.70,
    data_split_shuffle=True,
    ignore_features=[
        "attendance_current",
        "clientcode",
        "date_last_save",
        "date_last_send",
        "date_last_touch",
        "dimcustomermasterid",
        "events_current",
        "lkupclientid",
        "opens",
        "opens_prior",
        "seasonyear",
        "sends",
        "sends_prior",
        "spend_current",
        "volume_current"
    ],
    silent=True,
    verbose=False,
    categorical_features=[
        "product_last"
    ],
    numeric_features=[
        "atp_last",
        "attended_last",
        "attended_prior",
        "distance",
        "events_last",
        "events_prior",
        "tenure"
    ]
);

In [17]:
model_matrix = compare_models(
    fold=10,
    include=["rf"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9499,0.7693,0.3440,0.9250,0.9360,0.3567,0.3888,9.1020


In [20]:
best_model = create_model(model_matrix);
final_model = finalize_model(best_model);

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9501,0.7574,0.3427,0.9263,0.9363,0.3622,0.3938
1,0.9509,0.7790,0.3577,0.9257,0.9368,0.3675,0.4016
2,0.9489,0.7576,0.3354,0.9226,0.9348,0.3412,0.3735
3,0.9497,0.7797,0.3386,0.9242,0.9357,0.3574,0.3886
4,0.9505,0.7752,0.3519,0.9265,0.9368,0.3625,0.3963
5,0.9500,0.7733,0.3669,0.9260,0.9367,0.3701,0.3986
6,0.9497,0.7678,0.3295,0.9244,0.9356,0.3498,0.3834
7,0.9494,0.7619,0.3358,0.9249,0.9356,0.3482,0.3805
8,0.9500,0.7685,0.3524,0.9256,0.9363,0.3632,0.3940


### Let's load in our inference data and get retention scores using the model:

In [22]:
df_inference = helpers.get_product_propensity_dataset(
    cluster=CLUSTER,
    database=DATABASE,
    lkupclientid=LKUPCLIENTID,
    scoring_year=SCORING_YEAR,
    type_flag=1
)

df_inference.shape

Authorized as AROASQ4JELIXYLYV6P4UV:rkazmerik@stellaralgo.com


(387764, 25)